data/data.csv
有重複資料
562 => 522

In [104]:
import time

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/linear_features/cdhit80_k1_linear10_loc75.csv")

In [21]:
idx = np.arange(len(df))
np.random.shuffle(idx)
print(idx)
train_df = df[df.index.isin(idx[:int(len(idx) * 0.8)])]
print(len(train_df))
test_df = df[df.index.isin(idx[int(len(idx) * 0.8):])]
print(len(test_df))
print(train_df["loc"].value_counts())
print(test_df["loc"].value_counts())
len_box = [train_df['len'], test_df['len']]
plt.boxplot(len_box)
plt.show()

[ 691  376 1972 ... 2582  135  391]
2432
608


In [4]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
print(X.shape, y.shape)

(150, 4) (150,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.9666666666666667

In [28]:
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)

In [71]:
col = train_df.columns
x_train = train_df[col[:-1]]
y_train = train_df[col[-1]]
x_test = test_df[col[:-1]]
y_test = test_df[col[-1]]
print(x_train.shape)
print(y_train.shape)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
print(y_train.head(10))

(2432, 45)
(2432,)
0    Cytosolic
1    Cytosolic
2    Cytosolic
3    Cytosolic
4    Cytosolic
5    Cytosolic
6    Cytosolic
7    Cytosolic
8    Cytosolic
9    Cytosolic
Name: loc, dtype: object


In [72]:
idx = np.arange(len(x_train))
np.random.shuffle(idx)
train_x = x_train[x_train.index.isin(idx[:int(len(idx) * 0.8)])]
train_y = y_train[y_train.index.isin(idx[:int(len(idx) * 0.8)])]
vild_x = x_train[x_train.index.isin(idx[int(len(idx) * 0.8):])]
vild_y = y_train[y_train.index.isin(idx[int(len(idx) * 0.8):])]
print(len(train_y))
print(len(vild_y))

1945
487


In [74]:
clf = svm.SVC(kernel='linear', C=1).fit(train_x, train_y)
clf.score(vild_x, vild_y)

0.919917864476386

In [100]:
vild_nuc = vild_y == "Nucleus"
vild_cyt = vild_y == "Cytosolic"
score_nuc = clf.score(vild_x[vild_nuc], vild_y[vild_nuc])
score_cyt = clf.score(vild_x[vild_cyt], vild_y[vild_cyt])
print(score_nuc)
print(score_cyt)

0.02564102564102564
0.984375


In [102]:
clf = svm.SVC(kernel='linear', C=1, class_weight='balanced')
scores = cross_val_score(clf, train_x, train_y, cv=10)

In [101]:
clf = svm.SVC(kernel='linear', C=1, class_weight='balanced').fit(train_x, train_y)
clf.score(vild_x, vild_y)

0.4414784394250513

In [ ]:
start_time = time.time()
step_size = len(loader)
for epoch in range(epochs):
    for step, (batch_x, batch_mask, batch_y) in enumerate(loader):
        step_time = time.time()
        model.zero_grad()
        pred = model(batch_x)
        
        loss = loss_fn(pred.softmax(dim=1), batch_y)
        print('Epoch: %i | Step: %i/%i | Loss: %.2f | time: %.2f s' % (epoch, step, step_size, loss, time.time() - step_time))
        loss.backward()
        optimizer.step()

print('all time : ', time.time() - start_time,'s')